# Prueba 2 - Big Data

## Parte 2

### Francisca Pinto - Francisco Silva

### 29 de enero de 2022

Se importan las librerías necesarias para el procesamiento de los datos y la creación de los modelos correspondientes.

In [5]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import when, col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, GBTClassifier, DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

Se comprueba que el <code>SparkContext</code> está disponible:

In [6]:
sc

<SparkContext master=yarn appName=livy-session-0>

### Ejercicio 2 - Identificando la probabilidad de cierre de un servicio

> Todos los objetivos se deben resolver utilizando pyspark.

* Implemente el esquema de recodificación.
* Genere la recodificación del vector objetivo.
* Divida la muestra en conjuntos de entrenamiento (Preservando un 70% de los datos) y validación (preservando un 30% de los datos).
* Entrene tres modelos (LogisticRegression, GBTClassifier y DecisionTreeClassifier) sin modificar hiperparámetros que en base a los atributos recodificados del archivo business.json, clasifique aquellos servicios cerrados.
* Reporte cuál es el mejor modelo en base a la métrica AUC.
* Identifique cuales son los principales atributos asociados al cierre de un servicio.

Iniciamos la importación de los datos desde el bucket entregado, se imprime el esquema de datos y se toma una fila para corroborar la estructura.

In [7]:
df_yelp_business = spark.read.json('s3://bigdata-desafio/yelp-data/business.json')

In [8]:
df_yelp_business.printSchema()

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [9]:
df_yelp_business.take(1)

[Row(address='2818 E Camino Acequia Drive', attributes=Row(AcceptsInsurance=None, AgesAllowed=None, Alcohol=None, Ambience=None, BYOB=None, BYOBCorkage=None, BestNights=None, BikeParking=None, BusinessAcceptsBitcoin=None, BusinessAcceptsCreditCards=None, BusinessParking=None, ByAppointmentOnly=None, Caters=None, CoatCheck=None, Corkage=None, DietaryRestrictions=None, DogsAllowed=None, DriveThru=None, GoodForDancing=None, GoodForKids='False', GoodForMeal=None, HairSpecializesIn=None, HappyHour=None, HasTV=None, Music=None, NoiseLevel=None, Open24Hours=None, OutdoorSeating=None, RestaurantsAttire=None, RestaurantsCounterService=None, RestaurantsDelivery=None, RestaurantsGoodForGroups=None, RestaurantsPriceRange2=None, RestaurantsReservations=None, RestaurantsTableService=None, RestaurantsTakeOut=None, Smoking=None, WheelchairAccessible=None, WiFi=None), business_id='1SWheh84yJXfytovILXOAQ', categories='Golf, Active Life', city='Phoenix', hours=None, is_open=0, latitude=33.5221425, longit

Antes de iniciar el pre-procesamiento, se verán los valores del vector objetivo <code>is_open</code>:

In [10]:
df_yelp_business.groupBy('is_open').count().show()

+-------+------+
|is_open| count|
+-------+------+
|      0| 34084|
|      1|158525|
+-------+------+

> En la celda anterior se puede ver que el vector objetivo <code>is_open</code> ya se encuentra codificado de forma binaria, y en formato numérico.

De acuerdo al archivo <code>recording_business_schema</code> se tomarán las acciones siguientes:

1. <code>address</code>: atributo será eliminado.
2. <code>attributes</code>: atributo será eliminado.
    * <code>AcceptsInsurance</code>: atributo será re-codificado según las instrucciones.
    * <code>AgesAllowed</code>: atributo será re-codificado según las instrucciones.
    * <code>Alcohol</code>: atributo será re-codificado según las instrucciones.
    * <code>Ambience</code>: atributo será eliminado.
    * <code>BYOB</code>: atributo será eliminado.
    * <code>BYOBCorkage</code>: atributo será eliminado.
    * <code>BestNights</code>: atributo será eliminado.
    * <code>BikeParking</code>: atributo será eliminado.
    * <code>BusinessAcceptsBitcoin</code>: atributo será re-codificado según las instrucciones.
    * <code>BusinessAcceptsCreditCards</code>: atributo será eliminado.
    * <code>BusinessParking</code>: atributo será eliminado.
    * <code>ByAppointmentOnly</code>: atributo será eliminado.
    * <code>Caters</code>: atributo será eliminado.
    * <code>CoatCheck</code>: atributo será eliminado.
    * <code>Corkage</code>: atributo será eliminado.
    * <code>DietaryRestrictions</code>: atributo será eliminado.
    * <code>DogsAllowed</code>: atributo será re-codificado según las instrucciones.
    * <code>DriveThru</code>: atributo será eliminado.
    * <code>GoodForDancing</code>: atributo será eliminado.
    * <code>GoodForKids</code>: atributo será re-codificado según las instrucciones.
    * <code>GoodForMeal</code>: atributo será eliminado.
    * <code>HairSpecializesIn</code>: atributo será eliminado.
    * <code>HappyHour</code>: atributo será re-codificado según las instrucciones.
    * <code>HasTV</code>: atributo será re-codificado según las instrucciones.
    * <code>Music</code>: atributo será eliminado.
    * <code>NoiseLevel</code>: atributo será re-codificado según las instrucciones.
    * <code>Open24Hours</code>: atributo será eliminado.
    * <code>OutdoorSeating</code>: atributo será eliminado.
    * <code>RestaurantsAttire</code>: atributo será eliminado.
    * <code>RestaurantsCounterService</code>: atributo será eliminado.
    * <code>RestaurantsDelivery</code>: atributo será eliminado.
    * <code>RestaurantsGoodForGroups</code>: atributo será eliminado.
    * <code>RestaurantsPriceRange2</code>: atributo será re-codificado según las instrucciones.
    * <code>RestaurantsReservations</code>: atributo será eliminado.
    * <code>RestaurantsTableService</code>: atributo será eliminado.
    * <code>RestaurantsTakeOut</code>: atributo será eliminado.
    * <code>Smoking</code>: atributo será  re-codificado según las instrucciones.
    * <code>WheelchairAccessible</code>: atributo será eliminado.
    * <code>WiFi</code>: atributo será re-codificado según las instrucciones.
    * <code>business_id</code>: atributo será eliminado.
    * <code>categories</code>: atributo será  re-codificado según las instrucciones.
    * <code>city</code>: atributo será eliminado.
    * <code>hours</code>: atributo será eliminado.
        * <code>Friday</code>
        * <code>Monday</code>
        * <code>Saturday</code>
        * <code>Sunday</code>
        * <code>Thursday</code>
        * <code>Tuesday</code>
        * <code>Wednesday</code>
3. <code>is_open</code>: **vector objetivo**, se conserva y posteriormente se realizarán modificaciones correspondientes.
4. <code>latitude</code>: atributo será eliminado.
5. <code>longitude</code>: atributo será eliminado.
6. <code>name</code>: atributo será eliminado.
7. <code>postal_code</code>: atributo será eliminado.
8. <code>review_count</code>: atributo será eliminado.
9. <code>stars</code>: atributo será eliminado.
10. <code>state</code>: atributo será eliminado.

Pasos siguientes:

1. Se tomarán las re-codificaciones del archivo y se aplicarán sobre el dataframe en un loop.

2. Respecto al vector objetivo, la codificación según el enunciado es la siguiente:

    **"Usted deberá recodificarla de manera tal de identificar como 1 aquellos
    servicios que cerraron y 0 el resto"**.

    Por otro lado, la definición del diccionario en la web de <code>yelp</code> de <code>is_open</code> es:

    **"integer, 0 or 1 for closed or open, respectively"**.

    > Las definiciones son contrarias entre sí, por lo que se re-codificará intercambiando los valores y se renombrará como <code>is_closed</code>.

In [11]:
conditions = [
              when((col('attributes.AcceptsInsurance') == 'True')\
             | (col('attributes.AcceptsInsurance') == "\'True\'")\
             | (col('attributes.AcceptsInsurance') == "u\'True\'"), 1)\
              .otherwise(0),
              
              when((col('attributes.AgesAllowed') == 'allages')\
             | (col('attributes.AgesAllowed') == "\'allages\'")\
             | (col('attributes.AgesAllowed') == "u\'allages\'"), 1)\
              .otherwise(0),
              
              when((col('attributes.Alcohol') == 'beer_and_wine')\
             | (col('attributes.Alcohol') == "\'beer_and_wine\'")\
             | (col('attributes.Alcohol') == "u\'beer_and_wine\'")\
             | (col('attributes.Alcohol') == 'full_bar')\
             | (col('attributes.Alcohol') == "\'full_bar\'")\
             | (col('attributes.Alcohol') == "u\'full_bar\'"), 1)\
              .otherwise(0),
              
              when((col('attributes.BusinessAcceptsBitcoin') == 'True')\
             | (col('attributes.BusinessAcceptsBitcoin') == True)\
             | (col('attributes.BusinessAcceptsBitcoin') == "\'True\'")\
             | (col('attributes.BusinessAcceptsBitcoin') == "u\'True\'"), 1)\
              .otherwise(0),
              
              when((col('categories').rlike('Food'))\
             | (col('categories').rlike('Restaurants'))\
             | (col('categories').rlike('Bars')), 1)\
              .otherwise(0),
              
              when((col('categories').rlike('Banks'))\
             | (col('categories').rlike('Insurance'))\
             | (col('categories').rlike('Finance')), 1)\
              .otherwise(0),
              
              when((col('categories').rlike('Fitness'))\
             | (col('categories').rlike('Hospitals'))\
             | (col('categories').rlike('Health')), 1)\
              .otherwise(0),
              
              when((col('attributes.Smoking') == '\'yes\'')\
             | (col('attributes.Smoking') == 'u\'yes\'')\
             | (col('attributes.Smoking') == 'yes')\
             | (col('attributes.Smoking') == '\'outdoor\'')\
             | (col('attributes.Smoking') == 'u\'outdoor\'')\
             | (col('attributes.Smoking') == 'outdoor'), 1)\
              .otherwise(0),
              
              when((col('attributes.WiFi') == '\'free\'')\
             | (col('attributes.WiFi') == 'u\'free\'')\
             | (col('attributes.WiFi') == 'free'), 1)\
              .otherwise(0),
              
              when((col('attributes.RestaurantsPriceRange2') == 3)\
             | (col('attributes.RestaurantsPriceRange2') == 4), 1)\
              .otherwise(0),
              
              when((col('attributes.GoodForKids') == 'True')\
             | (col('attributes.GoodForKids') == True)\
             | (col('attributes.GoodForKids') == "\'True\'")\
             | (col('attributes.GoodForKids') == "u\'True\'"), 1)\
              .otherwise(0),
              
              when((col('attributes.HasTV') == 'True')\
             | (col('attributes.HasTV') == True)\
             | (col('attributes.HasTV') == "\'True\'")\
             | (col('attributes.HasTV') == "u\'True\'"), 1)\
              .otherwise(0),
              
              when((col('attributes.DogsAllowed') == 'True')\
             | (col('attributes.DogsAllowed') == True)\
             | (col('attributes.DogsAllowed') == "\True'\'")\
             | (col('attributes.DogsAllowed') == "u\'True\'"), 1)\
              .otherwise(0),
              
              when((col('attributes.NoiseLevel') == 'loud')\
             | (col('attributes.NoiseLevel') == "\'loud\'")\
             | (col('attributes.NoiseLevel') == "u\'loud\'")\
             | (col('attributes.NoiseLevel') == "very_loud")\
             | (col('attributes.NoiseLevel') == "\'very_loud\'")\
             | (col('attributes.NoiseLevel') == "u\'very_loud\'"), 1)\
              .otherwise(0),
              
              when((col('attributes.HappyHour') == 'True')\
             | (col('attributes.HappyHour') == True)\
             | (col('attributes.HappyHour') == "\'True\'")\
             | (col('attributes.HappyHour') == "u\'True\'"), 1)\
              .otherwise(0),
                
              when((col('is_open') == 0), 1)\
              .otherwise(0)
              ]

names = ['insurance',
         'all_ages_allowed',
         'alcohol_consumption',
         'bitcoin_friendly',
         'food_business',
         'finance_business',
         'health_business',
         'smokers',
         'free_wifi',
         'splurge',
         'kids_friendly',
         'has_tv',
         'dogs_friendly',
         'loud_place',
         'happy_hour',
         'is_closed']

In [12]:
for i in range(0, len(names)):
    df_yelp_business = df_yelp_business.withColumn(names[i], conditions[i])

Se revisarán los valores de los atributos re-codificados para saber si la aplicación del loop fue correcta. Además se inspeccionará el vector objetivo, que quedará con resultados opuestos a los mostrados anteriormente.

In [14]:
for name in names:
    df_yelp_business.groupBy(name).count().show()

+---------+------+
|insurance| count|
+---------+------+
|        1|  5277|
|        0|187332|
+---------+------+

+----------------+------+
|all_ages_allowed| count|
+----------------+------+
|               1|    18|
|               0|192591|
+----------------+------+

+-------------------+------+
|alcohol_consumption| count|
+-------------------+------+
|                  1| 26715|
|                  0|165894|
+-------------------+------+

+----------------+------+
|bitcoin_friendly| count|
+----------------+------+
|               1|   427|
|               0|192182|
+----------------+------+

+-------------+------+
|food_business| count|
+-------------+------+
|            1| 77332|
|            0|115277|
+-------------+------+

+----------------+------+
|finance_business| count|
+----------------+------+
|               1|  2204|
|               0|190405|
+----------------+------+

+---------------+------+
|health_business| count|
+---------------+------+
|              1| 21673|


Se obtendrán las columnas para dejar el dataframe solo con la información solicitada. <code>is_closed</code> ahora se llamará <code>label</code>, tal como solicitan los métodos que se ocuparán a continuación.

In [15]:
df_yelp_business.columns

['address', 'attributes', 'business_id', 'categories', 'city', 'hours', 'is_open', 'latitude', 'longitude', 'name', 'postal_code', 'review_count', 'stars', 'state', 'insurance', 'all_ages_allowed', 'alcohol_consumption', 'bitcoin_friendly', 'food_business', 'finance_business', 'health_business', 'smokers', 'free_wifi', 'splurge', 'kids_friendly', 'has_tv', 'dogs_friendly', 'loud_place', 'happy_hour', 'is_closed']

In [16]:
df_yelp_business = df_yelp_business.rdd.map(lambda row: (row['is_closed'],
                                                        row['review_count'],
                                                        row['stars'],
                                                        row['insurance'],
                                                        row['all_ages_allowed'],
                                                        row['alcohol_consumption'],
                                                        row['bitcoin_friendly'],
                                                        row['food_business'],
                                                        row['finance_business'],
                                                        row['health_business'],
                                                        row['smokers'],
                                                        row['free_wifi'],
                                                        row['splurge'],
                                                        row['kids_friendly'],
                                                        row['has_tv'],
                                                        row['dogs_friendly'],
                                                        row['loud_place'],
                                                        row['happy_hour']))\
                                    .toDF(['label',
                                           'review_count',
                                           'stars',
                                           'insurance',
                                           'all_ages_allowed',
                                           'alcohol_consumption',
                                           'bitcoin_friendly',
                                           'food_business',
                                           'finance_business',
                                           'health_business',
                                           'smokers',
                                           'free_wifi',
                                           'splurge',
                                           'kids_friendly',
                                           'has_tv',
                                           'dogs_friendly',
                                           'loud_place',
                                           'happy_hour',                                       
                                          ])

Se revisa el dataframe:

In [17]:
df_yelp_business.take(1)

[Row(label=1, review_count=5, stars=3.0, insurance=0, all_ages_allowed=0, alcohol_consumption=0, bitcoin_friendly=0, food_business=0, finance_business=0, health_business=0, smokers=0, free_wifi=0, splurge=0, kids_friendly=0, has_tv=0, dogs_friendly=0, loud_place=0, happy_hour=0)]

Se obtienen las columnas, se elimina el vector objetivo y se estructura la matriz de atributos y vector objetivo con <code>VectorAssembler</code>.

In [18]:
features = df_yelp_business.columns
features.remove('label')

In [26]:
af = VectorAssembler(inputCols = features,
                     outputCol = 'assembled_features')

af = af.transform(df_yelp_business)
af = af.select('label', 'assembled_features')

[Row(label=1, assembled_features=SparseVector(17, {0: 5.0, 1: 3.0})), Row(label=0, assembled_features=SparseVector(17, {0: 128.0, 1: 2.5, 4: 1.0, 6: 1.0, 12: 1.0, 15: 1.0}))]

In [32]:
af.show(5)

+-----+--------------------+
|label|  assembled_features|
+-----+--------------------+
|    1|(17,[0,1],[5.0,3.0])|
|    0|(17,[0,1,4,6,12,1...|
|    0|(17,[0,1,4,6,12,1...|
|    0|(17,[0,1,7],[3.0,...|
|    0|(17,[0,1],[4.0,4.0])|
+-----+--------------------+
only showing top 5 rows

Ahora se separarán las muestran de entrenamiento y validación, se entrenarán los modelos solicitados y se instanciará el evaluador con el método <code>BinaryClassificatorEvaluator</code>, que por defecto muestra la métrica <code>ROC AUC Score</code>.

In [33]:
train, test = af.randomSplit([0.7, 0.3])

In [34]:
model_logreg = LogisticRegression(featuresCol = 'assembled_features',
                                  labelCol = 'label',
                                  predictionCol = 'is_closed_pred')

model_logreg = model_logreg.fit(train)

model_gboostclassifier = GBTClassifier(featuresCol = 'assembled_features',
                                       labelCol = 'label',
                                       predictionCol = 'is_closed_pred')

model_gboostclassifier = model_gboostclassifier.fit(train)

model_dectreeclassifier = DecisionTreeClassifier(featuresCol = 'assembled_features',
                                                 labelCol = 'label',
                                                 predictionCol = 'is_closed_pred')

model_dectreeclassifier = model_dectreeclassifier.fit(train)

In [37]:
evaluator = BinaryClassificationEvaluator()
evaluator.setRawPredictionCol("is_closed_pred")

BinaryClassificationEvaluator_45bdc981acda

Se realizan listas para mostrar la evaluación en el caso de cada modelo.

In [36]:
model_names = ['Logistic Regression',
               'Gradient Boosting Classifier',
               'Decision Tree Classifier']

models = [model_logreg,
          model_gboostclassifier,
          model_dectreeclassifier]

for i in range(0, len(model_names)):
    print('AUC de modelo {}: {}'.format(model_names[i], evaluator.evaluate(models[i].transform(test))))

AUC de modelo Logistic Regression: 0.6934983848437615
AUC de modelo Gradient Boosting Classifier: 0.7065518747872895
AUC de modelo Decision Tree Classifier: 0.4399278993097432

El modelo con mejor comportamiento es el <code>Gradient Boosting Classifier</code>, del cual se extraerán los atributos con mayor peso con <code>.featureImportances</code>.

In [41]:
model_featureimportances = list(zip(features, list(model_gboostclassifier.featureImportances)))

In [47]:
model_featureimportances.sort(reverse = True, key = lambda x: x[1])

In [48]:
model_featureimportances

[('review_count', 0.2070665703534769), ('stars', 0.18719275222363202), ('food_business', 0.15740118954788632), ('alcohol_consumption', 0.08927422434600081), ('kids_friendly', 0.08434910305929749), ('insurance', 0.069833090154284), ('happy_hour', 0.042152203418428964), ('has_tv', 0.03385302027496311), ('splurge', 0.033732522998517436), ('free_wifi', 0.02247741590381045), ('finance_business', 0.01806441344951751), ('loud_place', 0.017118019250316845), ('smokers', 0.011230205482339078), ('dogs_friendly', 0.011195758262837814), ('health_business', 0.008974341589986222), ('all_ages_allowed', 0.005694019672355625), ('bitcoin_friendly', 0.0003911500123497122)]

Los atributos que tienen mayor influencia en los cálculos del algoritmo son:

1. <code>review_count</code>
2. <code>stars</code>
3. <code>food_business</code>
4. <code>alcohol_consumption</code>
5. <code>kids_friendly</code>